In [26]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
from decimal import Decimal, getcontext, ROUND_HALF_UP

logSH = pd.read_csv('/mnt/data/20201208/zt_6802/zt_6802_20201208_168.prevde/Logs_full/mdLog_SH_20201208_0904.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]
logSH1 = pd.read_csv('/mnt/data/20201208/zt_6802/zt_6802_20201208_168.30.6.170/Logs_full/mdLog_SH_20201208_0903.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]

# # 1. data format
print("original data source type: ")
print(logSH["source"].unique())
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

original data source type: 
[23 22]
new data source type: 
[31]
['09:04:19.000' '09:04:20.360' '09:04:21.070' ... '15:32:32.510'
 '15:32:32.000' '15:32:33.250']
['09:03:35.000' '09:03:36.000' '09:03:30.000' ... '15:32:08.000'
 '15:32:09.000' '15:32:11.000']


In [27]:
logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSH[(logSH["StockID"] == 600004) & (logSH["time"] >= 143000000) & (logSH["time"] <= 145000000)])
display(logSH1[(logSH1["StockID"] == 600004) & (logSH1["time"] >= 143000000) & (logSH1["time"] <= 145000000)])

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
9793166,1607409002028280,23996681,23,600004,SH,143001000,8254846,1.225330e+08,14.72,14.72,14.71,14.70,14.69,14.68,58790,90700,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11613
9793358,1607409002387219,23997133,22,600004,SH,143002390,8254946,1.225345e+08,14.73,14.72,14.71,14.70,14.69,14.68,59190,90700,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14500,3100,17200,4100,2100,15.06,11614
9795459,1607409005079669,24001597,23,600004,SH,143004000,8255346,1.225404e+08,14.73,14.72,14.71,14.70,14.69,14.68,59190,90300,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14100,3100,17200,4100,2100,15.06,11615
9795630,1607409005385287,24002127,22,600004,SH,143005390,8255346,1.225404e+08,14.73,14.72,14.71,14.70,14.69,14.68,59190,90300,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11615
9797643,1607409008095709,24006606,23,600004,SH,143007000,8256946,1.225639e+08,14.72,14.72,14.71,14.70,14.69,14.68,57690,90300,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10690345,1607410193482559,26124711,22,600004,SH,144953490,9898691,1.467001e+08,14.67,14.67,14.66,14.65,14.64,14.63,78727,42500,45600,10000,8300,14.68,14.69,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13869
10692056,1607410195743530,26128811,23,600004,SH,144955000,9898691,1.467001e+08,14.67,14.67,14.66,14.65,14.64,14.63,81227,42500,45600,10000,8300,14.68,14.69,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13869
10692761,1607410196480707,26130544,22,600004,SH,144956490,9898691,1.467001e+08,14.67,14.67,14.66,14.65,14.64,14.63,81327,42500,45600,10000,8300,14.68,14.69,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13869
10694445,1607410198781689,26134613,23,600004,SH,144958000,9899591,1.467133e+08,14.67,14.67,14.66,14.65,14.64,14.63,80427,42500,46300,10000,8300,14.68,14.69,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13873


,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
4828039,1607409002033234,88752613,31,600004,SH,143001000,8254846,122532999.0,14.72,14.72,14.71,14.70,14.69,14.68,58790,90700,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11613
4829153,1607409005083115,88770306,31,600004,SH,143004000,8255346,122540364.0,14.73,14.72,14.71,14.70,14.69,14.68,59190,90300,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14100,3100,17200,4100,2100,15.06,11615
4830271,1607409008099975,88789274,31,600004,SH,143007000,8256946,122563916.0,14.72,14.72,14.71,14.70,14.69,14.68,57690,90300,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11619
4831203,1607409010828138,88807762,31,600004,SH,143010000,8257046,122565388.0,14.72,14.72,14.71,14.70,14.69,14.68,58190,90300,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14600,2300,17200,4100,2100,15.06,11620
4832318,1607409013818064,88826778,31,600004,SH,143013000,8257046,122565388.0,14.72,14.72,14.71,14.70,14.69,14.68,58190,90300,323382,18500,53200,14.73,14.74,14.75,14.76,14.77,14600,2300,17200,4100,2100,15.06,11620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272659,1607410186768479,96233131,31,600004,SH,144946000,9893091,146617918.0,14.67,14.67,14.66,14.65,14.64,14.63,82527,42400,45600,10000,8300,14.68,14.69,14.70,14.71,14.72,30500,19800,22000,11700,10200,15.06,13862
5273851,1607410189792565,96254514,31,600004,SH,144949000,9895191,146648745.0,14.67,14.67,14.66,14.65,14.64,14.63,82427,42500,45600,10000,8300,14.68,14.69,14.70,14.71,14.72,30000,19200,22000,11700,10200,15.06,13864
5275040,1607410192783014,96276076,31,600004,SH,144952000,9896691,146670751.0,14.67,14.67,14.66,14.65,14.64,14.63,81227,42500,45600,10000,8300,14.68,14.69,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13867
5276226,1607410195748239,96296429,31,600004,SH,144955000,9898691,146700091.0,14.67,14.67,14.66,14.65,14.64,14.63,81227,42500,45600,10000,8300,14.68,14.69,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13869


In [28]:
# in_dex = logSH[logSH["source"] == 5]["StockID"].unique()
in_dex = [16, 300, 852, 905]
data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == 23)]
data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == 31)]

columns = ["StockID", "cum_volume", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", 'numTrades', 'openPrice']

data1.loc[data1['cum_volume'] > 0, 'openPrice'] = data1[data1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
data2.loc[data2['cum_volume'] > 0, 'openPrice'] = data2[data2['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')


data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())

print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
for cols in ['cum_amount']:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)
    print((n2-n1)/n1)

/home/work516/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1457
1459
4821972
4805681
4821973
-----------------------------------------------


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,cum_amount_y
2232,239265.0,1.607391e+15,334831.0,23.0,600683,SH,93000000.0,400,1.736000e+03,4.34,4.33,4.32,4.31,4.30,4.29,3000,10000,23000,14100,8800,4.34,4.37,4.39,4.40,4.43,2600,5000,4000,24000,20600,4.34,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2329,239579.0,1.607391e+15,340733.0,23.0,600192,SH,93000000.0,1400,7.154000e+03,5.11,5.10,5.09,5.08,5.07,5.06,6200,27000,10000,19300,22500,5.11,5.12,5.14,5.15,5.16,1600,11000,5000,11200,11200,5.11,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2331,239582.0,1.607391e+15,340736.0,23.0,603908,SH,93000000.0,500,1.380500e+04,27.64,27.64,27.51,27.40,27.36,27.27,200,300,500,500,1500,27.84,27.85,27.90,27.98,27.99,1200,200,500,900,1100,27.59,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2332,239583.0,1.607391e+15,340737.0,23.0,603110,SH,93000000.0,600,1.167000e+04,19.46,19.40,19.31,19.30,19.21,19.20,600,2000,3300,6000,2000,19.70,19.71,19.79,19.80,19.83,2100,2900,2000,100,100,19.44,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3727,242614.0,1.607391e+15,365000.0,23.0,600292,SH,93003000.0,3100,1.726600e+04,5.56,5.56,5.54,5.53,5.52,5.51,900,10000,17800,64600,6000,5.57,5.58,5.59,5.60,5.61,2500,3000,4100,49200,500,5.57,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4820478,11163396.0,1.607411e+15,27245591.0,23.0,603700,SH,145959000.0,2911959,8.293739e+07,28.68,28.69,0.00,0.00,0.00,0.00,46300,11400,0,0,0,28.69,0.00,0.00,0.00,0.00,46300,0,0,0,0,28.24,5373,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4820498,11163418.0,1.607411e+15,27245613.0,23.0,600089,SH,145959000.0,25713865,2.123109e+08,8.29,8.28,0.00,0.00,0.00,0.00,179300,37500,0,0,0,8.28,0.00,0.00,0.00,0.00,179300,0,0,0,0,8.24,15268,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4820502,11163422.0,1.607411e+15,27245617.0,23.0,600185,SH,145959000.0,31303321,2.543787e+08,8.11,8.08,0.00,0.00,0.00,0.00,236500,314019,0,0,0,8.08,0.00,0.00,0.00,0.00,236500,0,0,0,0,8.18,27696,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4820510,11163430.0,1.607411e+15,27245625.0,23.0,600406,SH,145959000.0,11210102,2.533782e+08,22.61,22.62,0.00,0.00,0.00,0.00,191300,0,0,0,0,22.62,0.00,0.00,0.00,0.00,191300,4668,0,0,0,22.43,23966,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0.003378700664375488


In [7]:
logSH[(logSH['source'] == 23) & (logSH['StockID'] == 688001) & (logSH['cum_volume'] == 857012)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
13281941,1607410804748990,115566097,23,688001,SH,150001000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13287878,1607410818970060,115707868,23,688001,SH,150016000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13288775,1607410826592100,115708765,23,688001,SH,150023000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13292000,1607410860009936,115711990,23,688001,SH,150056000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13293750,1607410878969307,115716054,23,688001,SH,150115000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13299090,1607410938963279,115723708,23,688001,SH,150215000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13304388,1607410998971840,115731320,23,688001,SH,150315000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13309679,1607411058973893,115738925,23,688001,SH,150415000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13316607,1607411118977383,115748167,23,688001,SH,150516000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794
13322019,1607411178979205,115755893,23,688001,SH,150615000,857012,34716100.4,40.32,40.32,...,40.38,40.4,40.41,374,200,447,300,300,40.9,1794


In [23]:
logSH[(logSH['StockID'] == 601727) & (logSH['cum_volume'] > 0)][['time', 'source'] + columns].head(20)

,time,source,StockID,cum_volume,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,numTrades,openPrice
99189,92501000,25,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,0.00
100476,92505000,25,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
100891,92515000,25,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
102516,92615000,25,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
104004,92715000,25,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
105492,92815000,25,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
106980,92915000,25,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
108304,93000000,25,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
109891,93002000,25,601727,196600,5.39,5.39,5.38,5.37,5.36,5.35,58900,20200,11600,131200,145700,5.40,5.41,5.42,5.43,5.44,49000,232200,146300,129800,51800,54,5.39
111430,93006000,25,601727,248000,5.39,5.39,5.38,5.37,5.36,5.35,33200,20200,12100,157500,160900,5.40,5.41,5.42,5.43,5.44,86100,235200,162800,133500,70800,80,5.39


In [25]:
logSH1[(logSH1['StockID'] == 601727) & (logSH1['cum_volume'] > 0)][['time', 'source'] + columns].head(30)

,time,source,StockID,cum_volume,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,numTrades,openPrice
224751,92501000,23,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,0.00
227269,92506060,22,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
227451,92505000,23,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,0.00
228464,92515000,23,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
229746,92529510,22,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
231946,92559180,22,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
233169,92615000,23,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
234171,92629190,22,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
236371,92659190,22,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39
237594,92715000,23,601727,143800,5.39,5.39,5.38,5.36,5.35,5.34,95500,19200,62800,130400,319400,5.40,5.41,5.42,5.43,5.44,30900,223100,82200,106800,39200,42,5.39


In [19]:
pd.set_option('max_rows', 100)
logSH1[(logSH1['StockID'] == 601113) & (logSH1['time'] >= 93003000)].head(10)[['StockID', 'source', 'time', 'cum_volume', 'cum_amount', 'bid1p', 'bid2p', 'bid1q', 'bid2q', 'ask1p', 'ask2p', 'ask1q', 'ask2q']]

,StockID,source,time,cum_volume,cum_amount,bid1p,bid2p,bid1q,bid2q,ask1p,ask2p,ask1q,ask2q
109531,601113,31,93003000,1202,4108.0,3.41,3.40,9998,29800,3.42,3.43,1900,6000
110385,601113,31,93006000,1202,4108.0,3.41,3.40,9998,29800,3.42,3.43,1900,62800
112581,601113,31,93009000,41000,139522.0,3.39,3.38,13000,100500,3.41,3.42,72002,1900
113779,601113,31,93012000,41000,139522.0,3.39,3.38,13000,100500,3.41,3.42,72002,1900
115182,601113,31,93015000,41000,139522.0,3.39,3.38,13000,100500,3.41,3.42,72002,28400
119451,601113,31,93024000,41000,139522.0,3.39,3.38,13000,100500,3.41,3.42,72002,28400
120822,601113,31,93027000,41000,139522.0,3.40,3.39,11100,13000,3.41,3.42,72002,28400
129302,601113,31,93046000,41000,139522.0,3.40,3.39,11100,15000,3.41,3.42,72002,28400
133830,601113,31,93056000,41000,139522.0,3.40,3.39,11100,15000,3.41,3.42,72002,28400
138636,601113,31,93106000,41000,139522.0,3.40,3.39,11100,15000,3.41,3.42,72002,28400


In [21]:
logSH[(logSH['StockID'] == 601113) & (logSH['time'] >= 93003000) & (logSH['source'] == 23)].head(10)[['StockID', 'source', 'time', 'cum_volume', 'cum_amount', 'bid1p', 'bid2p', 'bid1q', 'bid2q', 'ask1p', 'ask2p', 'ask1q', 'ask2q']]

,StockID,source,time,cum_volume,cum_amount,bid1p,bid2p,bid1q,bid2q,ask1p,ask2p,ask1q,ask2q
442473,601113,23,93003000,1202,4108.82,3.41,3.40,9998,29800,3.42,3.43,1900,6000
445399,601113,23,93006000,1202,4108.82,3.41,3.40,9998,29800,3.42,3.43,1900,62800
446349,601113,23,93006000,11200,38202.00,3.40,3.39,29800,13000,3.41,3.42,72002,1900
450028,601113,23,93009000,41000,139522.00,3.39,3.38,13000,100500,3.41,3.42,72002,1900
453280,601113,23,93012000,41000,139522.00,3.39,3.38,13000,100500,3.41,3.42,72002,1900
456702,601113,23,93015000,41000,139522.00,3.39,3.38,13000,100500,3.41,3.42,72002,28400
467026,601113,23,93024000,41000,139522.00,3.39,3.38,13000,100500,3.41,3.42,72002,28400
470391,601113,23,93027000,41000,139522.00,3.40,3.39,11100,13000,3.41,3.42,72002,28400
491175,601113,23,93046000,41000,139522.00,3.40,3.39,11100,15000,3.41,3.42,72002,28400
502351,601113,23,93056000,41000,139522.00,3.40,3.39,11100,15000,3.41,3.42,72002,28400


In [15]:
SH[(SH['StockID'] == 601113) & (SH['cum_volume'] <= 11200)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
1445,1606957303015911,3579,31,601113,SH,09:01:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
2930,1606957363014177,7197,31,601113,SH,09:02:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
4405,1606957423012722,10806,31,601113,SH,09:03:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
5885,1606957483014812,14418,31,601113,SH,09:04:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
7365,1606957543014318,18031,31,601113,SH,09:05:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
8825,1606957603013389,21624,31,601113,SH,09:06:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
10300,1606957663015885,25232,31,601113,SH,09:07:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
11808,1606957723015812,28873,31,601113,SH,09:08:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
13286,1606957783018665,32484,31,601113,SH,09:09:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0
14758,1606957843016257,36089,31,601113,SH,09:10:42.000,0,0.0,0.00,0.00,...,0.00,0.00,0.00,0,0,0,0,0,0.0,0


In [39]:
test[np.isnan(test["sequenceNo_y"])].head(100)

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,cum_amount_y
2254,439193,1606959028895576,2316224,23,603988,SH,93000000,600,8232.00,13.72,13.72,13.69,13.66,13.64,13.60,3100,8300,500,1000,5000,13.78,13.79,13.80,13.86,13.88,600,2000,800,800,13000,13.72,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2255,439194,1606959028895593,2316225,23,603887,SH,93000000,300,6123.00,20.41,20.38,20.30,20.28,20.23,20.22,1000,2500,900,500,200,20.40,20.49,20.50,20.52,20.55,1700,1300,1000,12500,1100,20.41,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3698,442909,1606959031952809,2755471,23,600365,SH,93003000,10000,23300.00,2.33,2.32,2.30,2.29,2.28,2.27,200,55700,86300,111900,2000,2.33,2.34,2.35,2.36,2.37,30300,118100,62700,61320,29700,2.33,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3699,443015,1606959031972134,2758162,23,603709,SH,93003000,1100,28025.00,25.48,25.33,25.30,25.27,25.26,25.25,1000,1300,200,1200,4400,25.46,25.51,25.64,25.65,25.66,4600,600,3500,1000,600,25.47,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5098,446349,1606959034544672,3123446,23,601113,SH,93006000,11200,38202.00,3.41,3.40,3.39,3.38,3.37,3.36,29800,13000,100500,1000,17000,3.41,3.42,3.43,3.44,3.45,72002,1900,62800,33800,2800,3.40,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5151,446804,1606959035529194,3264159,23,603351,SH,93007000,1000,23616.00,23.62,23.55,23.54,23.51,23.45,23.44,1500,100,200,1500,1300,23.61,23.62,23.66,23.67,23.68,8288,900,2000,1000,1700,23.60,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6472,449964,1606959037384189,3375499,23,603356,SH,93009000,12600,162511.00,12.90,12.89,12.87,12.83,12.82,12.81,400,11100,8000,10200,6100,12.90,12.95,12.96,12.97,12.98,9800,16600,23200,1700,4800,12.90,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7611,453202,1606959040138787,3478842,23,600356,SH,93012000,14400,123974.00,8.61,8.60,8.54,8.51,8.50,8.48,192600,600,1900,1100,7000,8.61,8.62,8.63,8.65,8.66,12400,2200,200,7500,21100,8.60,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8332,455758,1606959042837209,3575316,23,603040,SH,93014000,6100,205224.00,33.68,33.67,33.66,33.62,33.61,33.59,100,2500,2200,1900,1200,33.68,33.75,33.76,33.77,33.80,100,100,1200,200,600,33.61,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9262,457424,1606959044491863,3628814,23,603499,SH,93016000,4900,46409.00,9.47,9.46,9.45,9.43,9.42,9.41,900,1000,7000,700,6000,9.47,9.48,9.49,9.50,9.52,500,14000,13100,1500,1000,9.48,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
data2_1[(data2_1['StockID'] == 603003) & (data2_1['cum_volume'] >= 22200)].head(50)

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
17382,123909,1606959035204778,3486576,31,603003,SH,93035000,22200,182043.0,8.19,8.19,8.18,8.17,8.16,8.15,10200,2800,34600,500,28800,8.20,8.21,8.22,8.23,8.24,2900,10600,3900,6200,2300,8.2,31
18902,125495,1606959038520851,3568018,31,603003,SH,93038000,22200,182043.0,8.19,8.19,8.18,8.17,8.16,8.15,10300,2800,34600,500,28800,8.20,8.21,8.22,8.23,8.24,3300,10600,3900,6200,2300,8.2,31
20242,126895,1606959041523564,3638615,31,603003,SH,93041000,22200,182043.0,8.19,8.19,8.18,8.17,8.16,8.15,10300,2800,34600,500,28800,8.20,8.21,8.22,8.23,8.24,3300,10600,3600,6200,2300,8.2,31
21593,128307,1606959044513306,3709353,31,603003,SH,93044000,22200,182043.0,8.19,8.19,8.18,8.17,8.16,8.15,10300,2800,34600,500,28800,8.20,8.21,8.22,8.23,8.24,3300,10400,3600,6200,2300,8.2,31
22959,129729,1606959047523975,3779370,31,603003,SH,93047000,22200,182043.0,8.19,8.19,8.18,8.17,8.16,8.15,10400,2800,34600,500,28800,8.20,8.21,8.22,8.23,8.24,3300,10400,3400,6200,2300,8.2,31
24288,131132,1606959050515539,3845416,31,603003,SH,93050000,22200,182043.0,8.19,8.19,8.18,8.17,8.16,8.15,10400,2800,34600,500,28800,8.20,8.21,8.22,8.23,8.24,3500,10200,3400,6200,2300,8.2,31
25404,132307,1606959053179063,3903051,31,603003,SH,93053000,22200,182043.0,8.19,8.19,8.18,8.17,8.16,8.15,10400,2800,34600,2500,28800,8.20,8.21,8.22,8.23,8.24,3500,10200,3400,6200,2300,8.2,31
26804,133793,1606959056340119,3971828,31,603003,SH,93056000,22900,187783.0,8.20,8.19,8.18,8.17,8.16,8.15,10400,2800,34600,2500,28800,8.20,8.21,8.22,8.23,8.24,3000,10200,3400,6200,2300,8.2,35
28150,135216,1606959059342001,4035774,31,603003,SH,93059000,23000,188603.0,8.20,8.19,8.18,8.17,8.16,8.15,10400,2800,34600,2500,28800,8.20,8.21,8.22,8.23,8.24,2900,10200,3400,5200,2300,8.2,36
29272,136383,1606959062075526,4110117,31,603003,SH,93101000,23000,188603.0,8.20,8.19,8.18,8.17,8.16,8.15,10400,2800,34600,2500,28800,8.20,8.21,8.22,8.23,8.24,3200,10200,3400,5200,2300,8.2,36


In [51]:
logSH[(logSH['StockID'] == 603003) & (logSH['cum_volume'] <= 22700) & (logSH['source'] == 23)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
51485,1606956373666780,83868,23,603003,SH,84544000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
56833,1606956433666696,91529,23,603003,SH,84644000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
62166,1606956493670010,99175,23,603003,SH,84744000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
67484,1606956553671373,106806,23,603003,SH,84844000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
72847,1606956613677911,114482,23,603003,SH,84945000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
78165,1606956673669571,122113,23,603003,SH,85044000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
83483,1606956733674135,129744,23,603003,SH,85144000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
88816,1606956793670384,137390,23,603003,SH,85244000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
94149,1606956853670259,145036,23,603003,SH,85344000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0
99513,1606956913681649,152713,23,603003,SH,85444000,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0.0,0


In [13]:
pd.set_option('max_columns', 200)
data1_1[(data1_1['StockID'] == 603988) & (data1_1['cum_volume'] == 600)]

,index,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
2683,439193,1606959028895576,2316224,23,603988,SH,93000000,600,8232.0,13.72,13.72,13.69,13.66,13.64,13.6,3100,8300,500,1000,5000,13.78,13.79,13.8,13.86,13.88,600,2000,800,800,13000,13.72,3


In [29]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

3740.0
7993.0
13118.0
2597.0
3605.0
6744.0
0.1084824624194703
0.8915100464450401


In [33]:
in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]
# data2 = data2[data2["openPrice"] != 0]
columns = ["StockID", "cum_volume", "close", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

for cols in ["cum_amount"]:
    data1_1[cols] = data1_1[cols].round(1)
    data2_1[cols] = data2_1[cols].round(1)
    
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])

11400
11241
11400


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,bid1p_x,bid2p_x,bid3p_x,bid4p_x,bid5p_x,bid1q_x,bid2q_x,bid3q_x,bid4q_x,bid5q_x,ask1p_x,ask2p_x,ask3p_x,ask4p_x,ask5p_x,ask1q_x,ask2q_x,ask3q_x,ask4q_x,ask5q_x,openPrice,numTrades_x,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,cum_amount_y,bid1p_y,bid2p_y,bid3p_y,bid4p_y,bid5p_y,bid1q_y,bid2q_y,bid3q_y,bid4q_y,bid5q_y,ask1p_y,ask2p_y,ask3p_y,ask4p_y,ask5p_y,ask1q_y,ask2q_y,ask3q_y,ask4q_y,ask5q_y,numTrades_y
46,287781,1607391051209611,670654,22,852,SH,93051190,4067596,4.138115e+09,6734.4294,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,335109,1607391101240872,883773,22,852,SH,93141220,5604621,5.745572e+09,6735.2670,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,343912,1607391111180978,919602,22,852,SH,93151160,5853942,6.018171e+09,6735.3318,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,399657,1607391171153093,1150535,22,852,SH,93251130,7433379,7.719241e+09,6734.4912,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,509724,1607391291207000,1597707,22,852,SH,93451190,10560159,1.091498e+10,6745.0136,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10802,10451048,1607409891215896,25522533,22,852,SH,144451220,109275702,1.275901e+11,6737.7873,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10850,10498427,1607409951220261,25643950,22,852,SH,144551220,109805705,1.282206e+11,6739.1442,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10994,10640129,1607410131239772,25997956,22,852,SH,144851220,111550220,1.301009e+11,6736.0772,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11042,10688980,1607410191250897,26120868,22,852,SH,144951190,112246197,1.308340e+11,6734.5669,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,6735.2288,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
159 / 11400

0.013947368421052632

In [23]:
test[np.isnan(test["sequenceNo_x"])]

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount,close,bid1p_x,bid2p_x,bid3p_x,bid4p_x,bid5p_x,bid1q_x,bid2q_x,bid3q_x,bid4q_x,bid5q_x,ask1p_x,ask2p_x,ask3p_x,ask4p_x,ask5p_x,ask1q_x,ask2q_x,ask3q_x,ask4q_x,ask5q_x,openPrice,numTrades_x,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,bid1p_y,bid2p_y,bid3p_y,bid4p_y,bid5p_y,bid1q_y,bid2q_y,bid3q_y,bid4q_y,bid5q_y,ask1p_y,ask2p_y,ask3p_y,ask4p_y,ask5p_y,ask1q_y,ask2q_y,ask3q_y,ask4q_y,ask5q_y,numTrades_y
11400,NaN,NaN,NaN,NaN,300,NaN,NaN,672488,1.494403e+09,5029.1063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5029.1063,NaN,92147.0,1.607391e+15,1415711.0,31.0,SH,92521000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11401,NaN,NaN,NaN,NaN,852,NaN,NaN,1197304,1.172358e+09,6735.2288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6735.2288,NaN,92148.0,1.607391e+15,1415712.0,31.0,SH,92521000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11402,NaN,NaN,NaN,NaN,852,NaN,NaN,1261822,1.239191e+09,6735.4205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6735.2288,NaN,100480.0,1.607391e+15,1522667.0,31.0,SH,93001000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11403,NaN,NaN,NaN,NaN,300,NaN,NaN,694662,1.510133e+09,5029.0974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5029.1063,NaN,100481.0,1.607391e+15,1523715.0,31.0,SH,93001000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11404,NaN,NaN,NaN,NaN,300,NaN,NaN,1203956,2.578951e+09,5032.5394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5029.1063,NaN,102138.0,1.607391e+15,1952620.0,31.0,SH,93006000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18938,NaN,NaN,NaN,NaN,852,NaN,NaN,119482454,1.387434e+11,6734.4295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6735.2288,NaN,5509342.0,1.607411e+15,100819548.0,31.0,SH,150006000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18939,NaN,NaN,NaN,NaN,905,NaN,NaN,105424829,1.182563e+11,6423.5771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6424.1589,NaN,5509343.0,1.607411e+15,100819549.0,31.0,SH,150006000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18940,NaN,NaN,NaN,NaN,300,NaN,NaN,123073029,2.348806e+11,5009.8786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5029.1063,NaN,5510479.0,1.607411e+15,100887636.0,31.0,SH,150026000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18941,NaN,NaN,NaN,NaN,905,NaN,NaN,105494992,1.183249e+11,6423.6513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6424.1589,NaN,5510480.0,1.607411e+15,100887637.0,31.0,SH,150026000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
logSH1[(logSH1['StockID'] == 300) & (logSH1['cum_volume'] == 672488)]['cum_amount'].iloc[0]

1494402946.0

In [25]:
logSH[(logSH['StockID'] == 300) & (logSH['cum_volume'] == 672488)]['cum_amount'].iloc[0]

1494402946.2

In [15]:
test[test['sequenceNo_y'].isnull()]['time_x'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).astype(int)

count          159
mean     117010256
std       18193544
min       93051190
10%       94931196
20%      100711220
30%      102691190
40%      104671214
50%      110851220
60%      130431184
70%      132671220
80%      135571202
90%      141991214
max      145451220
Name: time_x, dtype: int64

In [43]:
test[(np.isnan(test["sequenceNo_y"])) & (test['StockID'] == 905) ]

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,...,ask2p_y,ask3p_y,ask4p_y,ask5p_y,ask1q_y,ask2q_y,ask3q_y,ask4q_y,ask5q_y,numTrades_y
6183,7944807,1606972140030343,83915334,22,905,SH,130832190,97140366,9.902394e+10,6454.1631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8863,10732587,1606975490178885,106898648,22,905,SH,140422130,123703375,1.248938e+11,6429.1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
test[(np.isnan(test["sequenceNo_y"])) & (test['StockID'] == 905) ][['StockID', 'source_x', 'time_x', 'cum_volume', 'cum_amount_x', 'close']]

,StockID,source_x,time_x,cum_volume,cum_amount_x,close
6183,905,22,130832190,97140366,9.902394e+10,6454.1631
8863,905,22,140422130,123703375,1.248938e+11,6429.1283


In [52]:
logSH1[(logSH1['StockID'] == 905) & (logSH1['time'] >= 130827000)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
3224912,1606972107250909,73624629,31,905,SH,130827000,97099139,9.898467e+10,6454.1631,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0
3228289,1606972117190923,73675683,31,905,SH,130837000,97163375,9.905053e+10,6454.0586,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0
3229310,1606972120141189,73691955,31,905,SH,130839000,97163375,9.905053e+10,6454.0586,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0
3230088,1606972122270012,73703547,31,905,SH,130842000,97198965,9.908572e+10,6453.2688,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0
3231732,1606972127246738,73736944,31,905,SH,130847000,97234978,9.912696e+10,6453.0459,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5724464,1606979920151273,115583976,31,905,SH,151838000,152214933,1.528918e+11,6440.8808,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0
5725948,1606979980151278,115587622,31,905,SH,151939000,152214933,1.528918e+11,6440.8808,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0
5727432,1606980040150774,115591277,31,905,SH,152040000,152214933,1.528918e+11,6440.8808,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0
5728915,1606980100149333,115594919,31,905,SH,152139000,152214933,1.528918e+11,6440.8808,0.0,...,0.0,0.0,0.0,0,0,0,0,0,6437.3843,0


In [38]:
logSH[(logSH['StockID'] == 852) & (logSH['time'] >= 93242000)].head()[['StockID', 'source', 'time', 'cum_volume', 'cum_amount', 'close']]

,StockID,source,time,cum_volume,cum_amount,close
620407,852,22,93242250,7686275,7.527288e+09,6710.3688
624557,852,22,93246160,7686275,7.527288e+09,6710.3688
625310,852,22,93247220,7813106,7.641059e+09,6711.1266
630915,852,22,93252190,7913812,7.751058e+09,6711.0108
636691,852,22,93257160,8077877,7.920631e+09,6711.0144


In [34]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

18266.25
42279.0
76680.75
8330.0
11949.0
15890.0
0.17792011386887288
0.8220798861311271


In [35]:
import pandas as pd
SH = pd.read_csv(r'/mnt/data/20201208/zt_6802/zt_6802_20201208_168.prevde/Logs_full/mdTradeLog_20201208_0904.csv',
                   encoding="utf-8", index_col=False)
SH1 = pd.read_csv(r'/mnt/data/20201208/zt_6802/zt_6802_20201208_168.30.6.170/Logs_full/mdTradeLog_20201208_0903.csv',
                   encoding="utf-8", index_col=False)
SH = SH[SH["exchId"] == 1]
SH1 = SH1[SH1["exchId"] == 1]

In [36]:
import numpy as np
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
# ####
# SH = SH[SH['TransactTime'] <= SH1['TransactTime'].max()]
# ####

columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum", 'secid', 'exchId']
# ####
# SH1 = SH1.drop_duplicates(columns, keep='first')
# ####

n1 = len(SH["SecurityID"].unique())
n2 = len(SH1["SecurityID"].unique())
print(n1)
print(n2)
print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))
print(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
if n1 != n2:
    sl = list(set(SH["SecurityID"].unique()) & set(SH1["SecurityID"].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

1457
1456
{511880}
set()
16192614
16192614
16192614
-----------------------------------------------


In [37]:
# speed check
re1 = re[~np.isnan(re["sequenceNo_x"])]
re1 = re1[~np.isnan(re1["sequenceNo_y"])]
diff = re1["clockAtArrival_y"] - re1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

2912.0
6437.0
12042.0
6034.0
8115.0
11800.0
0.059506389765111425
0.9404874963362926


In [6]:
stocks = SH["SecurityID"].unique()
m = 0
diff = []
for i in stocks:
    re1 = pd.merge(SH[SH["SecurityID"] == i], SH1[SH1["SecurityID"] == i], left_on=columns, right_on=columns, how="inner")
    if (sorted(re1["sequenceNo_y"]) != re1["sequenceNo_y"]).any() == False:
        m = m + 1
    else:
        diff.append(i)
if m == len(stocks):
    print("SHE Trade Data have same ordering within stocks")
else:
    print("SHE Trade Data don't have same ordering within stocks")

SHE Trade Data have same ordering within stocks
